
### Используя табличку user_actions_delta.parquet сгенерировать следующие признаки:


    1) Популярность по просмотрам
    2) Популярность по картэдам (количество добавлений в корзину)
    3) CTR - отношение количества картэдов к просмотрам
    4) Новизна - дата первого просмотра товара
    5) Среднее количество просмотров в день
    6) Среднее количество картэдов в день
    7) Количество просмотров в последний день
    8) Количество картэдов в последний день
    9) Отношение количества просмотров в последний день к среднему количеству просмотров в день
    10) Отношение 8 к 6
    11-12) Угловой коэффициент прямой, построенной по следующим точкам, по оси x день недели, по оси y количество просмотров/картэдов. Прямая строится методом МНК.


In [1]:
import numpy as np
import pandas as pd
import copy
import datetime as dt


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
cd /content/gdrive/My Drive/OZON

/content/gdrive/My Drive/OZON


## Считываение и предобработка данных

In [2]:
user_actions = pd.read_parquet('data/user_actions_august.parquet')
user_actions['clientid'] = user_actions['clientid'].astype('uint32')
user_actions['itemid'] = user_actions['itemid'].astype('uint32')
user_actions['action_type'] = user_actions['action_type'].map({'view': 0, 'to_cart': 1}).astype('bool')
user_actions['timestamp'] = user_actions['timestamp'].dt.date.apply(dt.datetime.toordinal).astype('uint32')

user_actions.head()

,clientid,itemid,action_type,timestamp
0,179153,135391270,False,737277
1,179153,135391272,False,737277
2,179153,135391270,False,737279
3,179153,142132354,False,737291
4,179153,140487634,False,737294


In [0]:
user_actions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57585220 entries, 0 to 57585219
Data columns (total 4 columns):
clientid       uint32
itemid         uint32
action_type    bool
timestamp      uint32
dtypes: bool(1), uint32(3)
memory usage: 713.9 MB


In [5]:
user_actions = pd.read_parquet('data/user_actions_august.parquet')
user_actions.head(10)

,clientid,itemid,action_type,timestamp
0,179153,135391270,view,2019-08-06 15:14:45
1,179153,135391272,view,2019-08-06 15:16:29
2,179153,135391270,view,2019-08-08 09:07:29
3,179153,142132354,view,2019-08-20 17:59:58
4,179153,140487634,view,2019-08-23 16:46:39
5,179153,137532475,view,2019-08-24 06:28:14
6,179153,140648413,view,2019-08-24 06:28:40
7,179153,140647829,view,2019-08-24 06:30:09
8,179153,147848390,to_cart,2019-08-24 07:51:54
9,179153,140647829,view,2019-08-24 06:29:22


### Считывание тестовой и тренеровочной выборки. Предобработка

In [6]:
test_target = pd.read_csv("data/target_test.csv")
train_target = pd.read_csv("data/target_train.csv")

In [6]:
test_target['clientid'] = test_target['clientid'].astype('uint32')
test_target['itemid'] = test_target['itemid'].astype('uint32')
test_target['jointitemid'] = test_target['jointitemid'].astype('uint32')
test_target['label'] = test_target['label'].astype('int8')
test_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 597158 entries, 0 to 597157
Data columns (total 5 columns):
clientid       597158 non-null uint32
itemid         597158 non-null uint32
jointitemid    597158 non-null uint32
label          597158 non-null int8
timestamp      597158 non-null object
dtypes: int8(1), object(1), uint32(3)
memory usage: 12.0+ MB


In [7]:
train_target['clientid'] = train_target['clientid'].astype('uint32')
train_target['itemid'] = train_target['itemid'].astype('uint32')
train_target['jointitemid'] = train_target['jointitemid'].astype('uint32')
train_target['label'] = train_target['label'].astype('int8')
train_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1390438 entries, 0 to 1390437
Data columns (total 5 columns):
clientid       1390438 non-null uint32
itemid         1390438 non-null uint32
jointitemid    1390438 non-null uint32
label          1390438 non-null int8
timestamp      1390438 non-null object
dtypes: int8(1), object(1), uint32(3)
memory usage: 27.8+ MB


## Генирация простых признаков

In [9]:
# Популярность
def popularity(data_action,data, action):
    if action == 'to_cart':
        id_count_Series = data_action[data_action['action_type']]['itemid'].value_counts()
    else:
        id_count_Series = data_action[~data_action['action_type']]['itemid'].value_counts()
    df = id_count_Series.rename_axis('jointitemid').reset_index(name= f'{action}_cnt')
    return pd.merge(data,df, how="left").fillna(0)

# CTR
def ctr(data, data_action):
    new_data = pd.DataFrame(data_action[['itemid', 'action_type']])
    viewed_data = new_data[new_data['action_type'] == False]['itemid'].value_counts()
    df1 = viewed_data.rename_axis('jointitemid').reset_index(name = 'view_cnt')
    to_cart_data = new_data[new_data['action_type'] == True]['itemid'].value_counts()
    df2 = to_cart_data.rename_axis('jointitemid').reset_index(name = 'to_cart_cnt')
    df = df1.join(df2.set_index('jointitemid'), on = 'jointitemid')
    new_df = df['to_cart_cnt'] / df['view_cnt']
    df['ctr'] = new_df
    df.drop(['view_cnt', 'to_cart_cnt'], axis = 1, inplace = True)
    return data.merge(df, how='left').fillna(0)

# Дата первого проссмотра
def date_first_view(data_action, data):
    id_item_series = data_action[~data_action['action_type']].groupby(['itemid'], sort=False)['timestamp'].min()
    df = id_item_series.rename_axis('jointitemid').reset_index(name='novelty_cnt')
    return data.merge(df, how="left").fillna(0)

# Среднее количество картэдов/просмотров в день
def day_avg_popularity(data_action, data, action):
    df = copy.copy(data_action)
    if action == 'to_cart':
        df_cart_countInDay_series = df[df['action_type']].groupby(['itemid','timestamp'])['itemid'].count()
    else:
        df_cart_countInDay_series = df[~df['action_type']].groupby(['itemid','timestamp'])['itemid'].count()
        
        
    df_cart_countInDay = df_cart_countInDay_series.rename_axis(['jointitemid','timestamp']).reset_index(name='count_day')
    
    df_cart_countInDay[f'{action}_day_avg_cnt'] = df_cart_countInDay.groupby(['jointitemid'])['count_day'].transform('mean')
    df_cart_countInDay.drop(['timestamp','count_day'], axis = 'columns', inplace=True)
    return data.merge(df_cart_countInDay.drop_duplicates(), how="left").fillna(0)

# Количество просмотров в последний день
def last_day_views(data_action, data):
    df = data_action.copy()
    data_views = df[df['action_type']]
    data_last_day_views = data_views[data_views['timestamp'] == max(data_views['timestamp'])]
    df_last_day = data_last_day_views.groupby(['itemid'])['timestamp'].size()
    new_df = df_last_day.rename_axis('jointitemid').reset_index(name='last_day_views')
    return data.merge(new_df, how='left').fillna(0)

#Количество картэдов в последний день
def cart_add_lust_day(data_action,data):
    df = copy.copy(data_action)
    data_addCart = df[df['action_type']]
    data_addCart_lustDay = data_addCart[data_addCart['timestamp']==max(data_addCart['timestamp'])]
    dS_lustDay = data_addCart_lustDay.groupby(['itemid'])['timestamp'].size()
    dataFrame = dS_lustDay.rename_axis('jointitemid').reset_index(name='to_cart_Lust_Day_cnt')
    return data.merge(dataFrame, how="left").fillna(0)

# Отношение количества просмотров в последний день к среднему количеству просмотров в день
def relation_last_mean(data_action, data):
    df = pd.DataFrame()
    quantity_ldviews = last_day_views(data_action, data)
    mean_vpday = day_avg_popularity(data_action, data, 'view')
    df['jointitemid'] = mean_vpday['itemid']
    df['relation_ldv_dap'] = quantity_ldviews['last_day_views'] / mean_vpday['view_day_avg_cnt']
    return data.merge(df, how='left').fillna(0)

# Отношение 8 к 6
def relation_lust_mean(data_action,data):
    ca_Ld = cart_add_lust_day(data_action,data)
    ca_Md = day_avg_popularity(data_action,data,'to_cart')
    datafr = pd.DataFrame()
    datafr["jointitemid"] =ca_Md["itemid"]
    datafr["lastDay/dayAvg__cnt"]=ca_Ld['to_cart_Lust_Day_cnt']/ca_Md['to_cart_day_avg_cnt']
    return data.merge(datafr, how="left").fillna(0)


def generate_popularity_features(user_actions, target):
    res = popularity(user_actions,target, 'view')
    res = popularity(user_actions,res, 'to_cart')
    # res = ctr(user_action, res)
    res = date_first_view(user_actions, res)
    res = day_avg_popularity(user_actions, res,'to_cart')
    res = day_avg_popularity(user_actions, res,'view')
    res = last_day_views(user_actions, res)
    res = cart_add_lust_day(user_actions, res)
    res = relation_last_mean(user_actions, res)
    res = relation_lust_mean(user_actions, res)
    return res

In [0]:
# # пример вызова одной функции
# day_avg_popularity(user_actions,train_target, 'view')

In [14]:
ctr(train_target, user_actions)

,clientid,itemid,jointitemid,label,timestamp,ctr
0,7833842,31499843,138176581,1,2019-09-07 20:11:01,0.290323
1,19548158,147389610,148381589,0,2019-08-31 22:32:31,0.375000
2,32943407,6261257,4490956,0,2019-09-06 15:19:30,0.000000
3,10185243,148455169,148455173,0,2019-09-03 17:43:50,0.100000
4,30552232,152440009,152440052,0,2019-09-01 07:07:59,0.111111
...,...,...,...,...,...,...
1390433,7288237,148472244,148468962,0,2019-09-05 14:21:19,0.000000
1390434,10220324,149804536,150514034,0,2019-09-02 14:27:26,0.000000
1390435,20369720,151359851,149672279,0,2019-09-02 09:43:45,0.485255
1390436,15452247,136264892,26040639,0,2019-09-03 19:35:37,0.176471


In [0]:
train = generate_popularity_features(user_actions, train_target)
train.head()


In [0]:
%%time
test = generate_popularity_features(user_actions,test_target)
test

CPU times: user 1min 8s, sys: 27 s, total: 1min 35s
Wall time: 1min 50s


,clientid,itemid,jointitemid,label,timestamp,view_cnt,to_cart_cnt,novelty_cnt,to_cart_day_avg_cnt,view_day_avg_cnt,to_cart_Lust_Day_cnt,lastDay/dayAvg__cnt
0,8081929,152898248,152875664,0,2019-09-06 18:57:23,10.0,2.0,737276.0,1.000000,2.000000,0.0,0.000000
1,33378638,144847078,140715321,0,2019-09-02 07:09:31,211.0,19.0,737272.0,1.727273,7.814815,1.0,0.000000
2,33378638,144847078,140715321,0,2019-09-02 07:09:31,211.0,19.0,737272.0,1.727273,7.814815,1.0,0.000000
3,33378638,144847078,140715321,0,2019-09-02 07:09:31,211.0,19.0,737272.0,1.727273,7.814815,1.0,0.000000
4,33378638,144847078,140715321,0,2019-09-02 07:09:31,211.0,19.0,737272.0,1.727273,7.814815,1.0,0.652174
5,33378638,144847078,140715321,0,2019-09-02 07:09:31,211.0,19.0,737272.0,1.727273,7.814815,1.0,0.000000
6,33378638,144847078,140715321,0,2019-09-02 07:09:31,211.0,19.0,737272.0,1.727273,7.814815,1.0,0.600000
7,36237195,149717596,149390783,0,2019-09-01 07:14:01,282.0,20.0,737272.0,1.666667,9.096774,1.0,0.000000
8,36237195,149717596,149390783,0,2019-09-01 07:14:01,282.0,20.0,737272.0,1.666667,9.096774,1.0,0.000000
9,36237195,149717596,149390783,0,2019-09-01 07:14:01,282.0,20.0,737272.0,1.666667,9.096774,1.0,0.487805


### Рассчет углового коэфициента мнк

In [41]:
# Угловой коэффициент прямой, построенной по следующим точкам, по оси x день недели, по оси y количество картэдов. Прямая строится методом МНК.
# реализуем функцию, которая считает коэфициент для одного товара
def get_coef(df,action):
    x = df.day.values # np.array иксов
    y = df['count_'+action].values #np.array игриков
    if (7 * (x**2).sum() - x.sum()**2) == 0:
        return 0
    k = (7 * (x * y).sum() - x.sum()*y.sum()) / (7 * (x**2).sum() - x.sum()**2)
    return k

In [20]:
user_actions

,clientid,itemid,action_type,timestamp
0,179153,135391270,False,737277
1,179153,135391272,False,737277
2,179153,135391270,False,737279
3,179153,142132354,False,737291
4,179153,140487634,False,737294
5,179153,137532475,False,737295
6,179153,140648413,False,737295
7,179153,140647829,False,737295
8,179153,147848390,True,737295
9,179153,140647829,False,737295


In [35]:
def count_vewCart_7day(user_actions, action):
    if action == 'view':
        condition = (user_actions['timestamp'] > (user_actions['timestamp'].max() - 7)) & (user_actions['action_type']==False)
    else:
        condition = (user_actions['timestamp'] > (user_actions['timestamp'].max() - 7)) & (user_actions['action_type']==True)

    # создаем две таблички all_items - все товары, days - номер дня
    all_items = user_actions[['itemid']].drop_duplicates()
    days = pd.DataFrame({'day': [0, 1, 2, 3, 4, 5, 6]})

    # соединим эти таблички, каждому айтему приставим табличку с днями

    all_items['key'] = 1
    days['key'] = 1
    item_day_df = all_items.merge(days, on='key').drop(columns=['key'])


    cnts_df = (
        user_actions[condition]
        .groupby(['itemid', 'timestamp'])['clientid']
        .agg(['count'])
        .reset_index()
    )

    cnts_df['day'] = cnts_df['timestamp'] - cnts_df['timestamp'].min()

    # соединим табличку с товаром и номером дня с табличкой каунтов
    # пропуски заполним нулями

    item_day_df = (
        item_day_df
        .merge(cnts_df.drop(columns=['timestamp']), on = ['itemid','day'], how='left')
        .fillna(0)
    )
    item_day_df = item_day_df.rename(columns={'count': 'count_'+action})
    return item_day_df

In [39]:
%%time
item_day_view = count_vewCart_7day(user_actions, 'view')

CPU times: user 24.5 s, sys: 10 s, total: 34.5 s
Wall time: 40.1 s


In [37]:
%%time
item_day_cart = count_vewCart_7day(user_actions, 'to_cart')

CPU times: user 19.4 s, sys: 9.07 s, total: 28.5 s
Wall time: 34 s


In [40]:
item_day_view 

,itemid,day,count_view
0,135391270,0,0.0
1,135391270,1,2.0
2,135391270,2,1.0
3,135391270,3,0.0
4,135391270,4,0.0
5,135391270,5,0.0
6,135391270,6,1.0
7,135391272,0,2.0
8,135391272,1,0.0
9,135391272,2,0.0


In [46]:
def mnk_coef(item_day, action):
    # последний шаг в цикле вычисляем для всех товаров необходимый коэффициент
    dfs = []
    ks = []
    items = []
    for group_name, group in item_day.groupby('itemid'):
        k = get_coef(group, action)
        items.append(group_name)
        ks.append(k)
    return pd.DataFrame({'itemid': items, 'k_'+action: ks})

In [47]:
%%time
k_tocart = mnk_coef(item_day_cart, 'to_cart')

CPU times: user 14min 6s, sys: 12.4 s, total: 14min 18s
Wall time: 16min 26s


In [49]:
%%time
k_view = mnk_coef(item_day_view , 'view')

CPU times: user 13min 43s, sys: 10.9 s, total: 13min 54s
Wall time: 15min 25s


In [50]:
k_view

,itemid,k_view
0,63859,0.000000
1,63934,0.000000
2,63958,-0.071429
3,63970,0.107143
4,63979,0.000000
5,63988,0.000000
6,63991,0.000000
7,63997,0.000000
8,64004,0.000000
9,64055,0.000000


In [51]:
train_target = pd.read_csv('data/train_with_scores.csv')
test_target = pd.read_csv('data/test_with_scores.csv')

In [58]:
train = pd.merge(train_target, k_view, on='itemid', how = 'left').drop_duplicates()
train = pd.merge(train, k_tocart, on='itemid', how = 'left').drop_duplicates()
train = train.fillna(0)

In [59]:
test = pd.merge(test_target, k_view, on='itemid', how = 'left').drop_duplicates()
test = pd.merge(test, k_tocart, on='itemid', how = 'left').drop_duplicates()
test = test.fillna(0)

## запись в файлы

In [61]:
test.to_csv("data/test_with_scores.csv", index=False)
train.to_csv("data/train_with_scores.csv", index=False)